## Setting Up Data

Copied to local folder and set later root_folder variable according to it

Install also scipy. Indirect use.

In [ ]:
root_folder = '/home/juha/PycharmProjects/Sandbox_Deep-learning-using-Tensorflow-Lite-on-Raspberry-Pi/Project_2: Visual Calculator'
path_to_data = root_folder + '/data/extracted_images/'

## Data Processing Pipeline

#### Data generators

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing import image
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
data_gen = ImageDataGenerator(shear_range=0.2,zoom_range=0.2,rescale=1./255 ,validation_split=0.2)

#path_to_data = '/data/extracted_images/'

training_dataset = data_gen.flow_from_directory(path_to_data,(128,128),subset="training" ,color_mode='grayscale')

validation_dataset = data_gen.flow_from_directory(path_to_data,(128,128),subset="validation",color_mode='grayscale')


### Exploring Our Dataset

In [ ]:
print(training_dataset.class_indices)
labels = ["divide" , "eight","five","four","min","mul","nine","one","plus","seven","six","three","two"]

### Visualize Data

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 6, figsize=(20,20))
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
training_images, _ = next(training_dataset)
plotImages(training_images[:6])

## Implementing the Model

In [ ]:
# initialising the CNN
visual_calculator_model = Sequential()

# convolution to extract features from images
visual_calculator_model.add(Conv2D(32, (3, 3), input_shape = (128, 128, 1), activation = 'relu'))

# max pooling to get max / largest values in feature map
# down sampling technique to get the most present features
visual_calculator_model.add(MaxPooling2D(pool_size = (2, 2)))

# more convolution and max pooling layers
visual_calculator_model.add(Conv2D(64, (3, 3), activation = 'relu'))
visual_calculator_model.add(MaxPooling2D(pool_size = (2, 2)))
visual_calculator_model.add(Conv2D(128, (3, 3), activation = 'relu'))
visual_calculator_model.add(MaxPooling2D(pool_size = (2, 2)))
visual_calculator_model.add(Conv2D(256, (3, 3), activation = 'relu'))
visual_calculator_model.add(MaxPooling2D(pool_size = (2, 2)))

# flattening is converting the data into a 1-dimensional array
visual_calculator_model.add(Flatten())
visual_calculator_model.add(Dense(units = 1024, activation = 'relu'))
visual_calculator_model.add(Dense(units = 13, activation = 'softmax'))

# compiling the CNN
visual_calculator_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# show summary of the created model
visual_calculator_model.summary()

In [ ]:
history = visual_calculator_model.fit(training_dataset , epochs =15 ,validation_data= validation_dataset, callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2) )

In [ ]:
# Note: '"/content' replaced to 'root_folder +"'
# # Legacy to be replaced: 
# current warning is visual_calculator_model.save_weights(root_folder +"/content/vc_model.weights.h5")
visual_calculator_model.save(root_folder +"/vc_model.h5")
#visual_calculator_model.save(root_folder + "/vc_model.keras")

In [ ]:
visual_calculator_model.save_weights(root_folder +"/vc_model.weights.h5")

## Testing model

Prediction - Accuracy

In [ ]:
testing_image = image.load_img(root_folder +'/data/extracted_images/nine/10.jpg', color_mode="grayscale",target_size=(128,128))
plt.imshow(testing_image)

testing_image = image.img_to_array(testing_image)
print(testing_image.dtype)
print(testing_image.shape)

testing_image = np.expand_dims(testing_image , axis=0)
print(testing_image.shape)
prediction_result = visual_calculator_model.predict(testing_image)
print("Model Predictions :", labels[np.argmax(prediction_result)] , "\n" )

LOSS - Validation Metrices

In [ ]:
history.history

In [ ]:
loss_plotter = plt.figure(figsize=(18,5)) 
mse_plot  = loss_plotter.add_subplot(121)
mse_plot_1  = loss_plotter.add_subplot(122)

mse_plot_1.plot(history.epoch,history.history["loss"],history.history["val_loss"] )
mse_plot_1.set_xlabel("Epochs")
mse_plot_1.set_ylabel('Errors')
mse_plot_1.legend(["loss","val_loss"])


mse_plot.plot(history.epoch,history.history["accuracy"],history.history["val_accuracy"])
mse_plot.set_xlabel("Epochs")
mse_plot.set_ylabel('Accuracy')
mse_plot.legend(["accuracy","val_accuracy"])

## TF Lite

Conversion

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(visual_calculator_model)
converter.optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
tflite_model = converter.convert()

In [ ]:
open(root_folder +"/vc_model.tflite","wb").write(tflite_model)

Size Comparison

In [ ]:
# print(f"Root folder: {root_folder}")
open(root_folder +"/vc_model.tflite","wb").write(tflite_model)
print("Main Model Size :" , round((os.path.getsize(root_folder +"/vc_model.h5"))/(1024*1024) ,3 ) , "MB")
print("Lite Model Size :" , round((os.path.getsize(root_folder +"/vc_model.tflite"))/(1024*1024),3 ), "MB"  )

TF Lite Prediction

In [ ]:
interpreter = tf.lite.Interpreter(root_folder +'/vc_model.tflite')

input_details   = interpreter.get_input_details()
output_details = interpreter.get_output_details()


print(input_details)

print("-"*10)
print(output_details)
print("-"*10)
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'] , testing_image)
interpreter.invoke()

tflite_prediction_result = interpreter.get_tensor(output_details[0]['index'])
print("Lite Model Predictions :", labels[np.argmax(tflite_prediction_result)] , "\n" )